# Abstract

*A problem well stated is a problem half-solved.*

*This is your space to describe your intentions for the project, before writing a single line of code. What are you studying? What are you hoping to build? If you can't explain that clearly before you start digging into the data, you're going to have a hard time planning where to go with this.*

# Obtain the Data

*Describe your data sources here and explain why they are relevant to the problem you are trying to solve.*

*Your code should download the data and save it in data/raw. If you've got the data from an offline source, describe where it came from and what the files look like. Don't do anything to the raw data files just yet; that comes in the next step.*

*After completing this step, be sure to edit `references/data_dictionary` to include descriptions of where you obtained your data and what information it contains.*

In [9]:
## %%writefile ../src/data/make_dataset.py

# Imports
import boto3
import requests
from datetime import date, timedelta

# Helper functions go here
def s3_file_exists(filename, s3, bucket_name):
    bucket = s3.Bucket(bucket_name)
    key = filename
    objs = list(bucket.objects.filter(Prefix=key))
    return len(objs) > 0 and objs[0].key == key


def save_to_s3(url, filepath, s3, bucket_name):
    # Do this as a quick and easy check to make sure your S3 access is OK
    bucket_found = bucket_name in [b.name for b in s3.buckets.all()]
    if not bucket_found:
        print('Not seeing your S3 bucket, might want to double check permissions in IAM')

    # Given an Internet-accessible URL, download the image and upload it to S3,
    # without needing to persist the image to disk locally
    r = requests.get(url, stream=True)
    if r.headers['Content-Type'] == 'image/jpeg':
        file_object = r.raw
        data = file_object.read()

        # Do the actual upload to s3
        s3.Bucket(bucket_name).put_object(Key=filepath, Body=data)
    else:
        print(f"No image found at {url}")
    return


def get_thumbnail_paths(start_date, end_date):
    base = "https://spaceneedledev.com/panocam/assets/"
    tail = "thumbnail.jpg"
    get_url = lambda x: f'https://spaceneedledev.com/panocam/assets/{x}'

    num_days = (end_date - start_date).days
    date_list = [end_date - timedelta(days=x) for x in range(0, num_days)]
    times = [str(t).zfill(6) for t in range(211000, 50000-1, -1000) if int(str(t)[-4]) < 6]
    
    paths = []
    for day in date_list:
        y = day.year
        m = str(day.month).zfill(2)
        d = str(day.day).zfill(2)
        
        for t in times:
            filepath = f'{y}/{m}/{d}/{y}_{m}{d}_{t}/{tail}'
            url = get_url(filepath)
            paths.append((url, filepath))
    return paths


def get_thumbnails():
    s3 = boto3.resource('s3')
    bucket_name = 'eye-of-the-needle'
    bucket = s3.Bucket(bucket_name)
    files = [file.key for file in bucket.objects.filter()]
    
    start_date = date(2015, 1, 1)
    end_date = date(2017, 6, 30)
    paths = get_thumbnail_paths(start_date, end_date)
    
    for path in paths:
        if path[1] not in files:
            save_to_s3(path[0], path[1], s3, bucket_name)
    
    return


def run():
    """
    Executes a set of helper functions that download data from one or more sources
    and saves those datasets to the data/raw directory.
    """
    get_thumbnails()
    return

In [ ]:
run()

No image found at https://spaceneedledev.com/panocam/assets/2017/06/29/2017_0629_100000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/29/2017_0629_095000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/28/2017_0628_054000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/28/2017_0628_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/27/2017_0627_055000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/27/2017_0627_054000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/27/2017_0627_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/26/2017_0626_155000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/26/2017_0626_154000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/26/2017_0626_0

No image found at https://spaceneedledev.com/panocam/assets/2017/06/06/2017_0606_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/04/2017_0604_060000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/04/2017_0604_055000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/04/2017_0604_054000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/04/2017_0604_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/03/2017_0603_055000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/03/2017_0603_054000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/03/2017_0603_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/02/2017_0602_060000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/06/02/2017_0602_0

No image found at https://spaceneedledev.com/panocam/assets/2017/05/02/2017_0502_201000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/05/02/2017_0502_060000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/05/01/2017_0501_201000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/05/01/2017_0501_095000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/05/01/2017_0501_094000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/05/01/2017_0501_062000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/05/01/2017_0501_061000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/30/2017_0430_161000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/30/2017_0430_160000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/30/2017_0430_0

No image found at https://spaceneedledev.com/panocam/assets/2017/04/14/2017_0414_051000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/14/2017_0414_050000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/13/2017_0413_211000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/13/2017_0413_210000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/13/2017_0413_064000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/13/2017_0413_051000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/13/2017_0413_050000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/12/2017_0412_211000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/12/2017_0412_210000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/12/2017_0412_1

No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_205000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_204000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_193000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_171000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_170000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_052000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_051000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/02/2017_0402_050000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/04/01/2017_0401_2

...

No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_203000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_202000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_201000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_200000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_195000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_194000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_193000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_192000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_191000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/06/2017_0306_0

No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_200000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_195000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_194000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_193000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_192000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_191000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_190000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_174000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_072000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/03/02/2017_0302_0

No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_070000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_065000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_064000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_063000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_062000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_061000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_060000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_055000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_054000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/28/2017_0228_0

No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_210000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_205000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_204000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_203000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_202000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_201000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_200000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_195000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_194000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/02/25/2017_0225_1

No image found at https://spaceneedledev.com/panocam/assets/2017/02/22/2017_0222_185000/thumbnail.jpg


In [5]:
# Uses the creds in ~/.aws/credentials
s3 = boto3.resource('s3')
bucket_name_to_upload_image_to = 'eye-of-the-needle'
folder = 'thumbnails/'
s3_image_filename = 'test_s3_image.jpg'
filepath = folder + s3_image_filename
internet_image_url = 'https://spaceneedledev.com/panocam/assets/2019/05/28/2019_0528_120000/thumbnail.jpg'

# Do this as a quick and easy check to make sure your S3 access is OK
for bucket in s3.buckets.all():
    if bucket.name == bucket_name_to_upload_image_to:
        print('Good to go. Found the bucket to upload the image into.')
        good_to_go = True

if not good_to_go:
    print('Not seeing your s3 bucket, might want to double check permissions in IAM')

# Given an Internet-accessible URL, download the image and upload it to S3,
# without needing to persist the image to disk locally
req_for_image = requests.get(internet_image_url, stream=True)
file_object_from_req = req_for_image.raw
req_data = file_object_from_req.read()

# Do the actual upload to s3
s3.Bucket(bucket_name_to_upload_image_to).put_object(Key=filepath, Body=req_data)

Good to go. Found the bucket to upload the image into.


s3.Object(bucket_name='eye-of-the-needle', key='thumbnails/test_s3_image.jpg')

In [180]:
def s3_file_exists(filename, s3, bucket_name):
    bucket = s3.Bucket(bucket_name)
    key = filename
    objs = list(bucket.objects.filter(Prefix=key))
    return len(objs) > 0 and objs[0].key == key


def save_to_s3(url, filepath, s3, bucket_name):
    # Do this as a quick and easy check to make sure your S3 access is OK
    bucket_found = bucket_name in [b.name for b in s3.buckets.all()]
    if not bucket_found:
        print('Not seeing your S3 bucket, might want to double check permissions in IAM')

    # Given an Internet-accessible URL, download the image and upload it to S3,
    # without needing to persist the image to disk locally
    r = requests.get(url, stream=True)
    if r.headers['Content-Type'] == 'image/jpeg':
        file_object = r.raw
        data = file_object.read()

        # Do the actual upload to s3
        s3.Bucket(bucket_name).put_object(Key=filepath, Body=data)
    else:
        print(f"No image found at {url}")
    return


def get_thumbnail_paths(start_date, end_date):
    base = "https://spaceneedledev.com/panocam/assets/"
    tail = "thumbnail.jpg"
    get_url = lambda x: f'https://spaceneedledev.com/panocam/assets/{x}'

    num_days = (end_date - start_date).days
    date_list = [end_date - timedelta(days=x) for x in range(0, num_days)]
    times = [str(t).zfill(6) for t in range(211000, 50000-1, -1000) if int(str(t)[-4]) < 6]
    
    paths = []
    for day in date_list:
        y = day.year
        m = str(day.month).zfill(2)
        d = str(day.day).zfill(2)
        
        for t in times:
            filepath = f'{y}/{m}/{d}/{y}_{m}{d}_{t}/{tail}'
            url = get_url(filepath)
            paths.append((url, filepath))
    return paths


def get_thumbnails():
    s3 = boto3.resource('s3')
    bucket_name = 'eye-of-the-needle'
    bucket = s3.Bucket(bucket_name)
    files = [file.key for file in bucket.objects.filter()]
    
    start_date = date(2015, 1, 1)
    end_date = date.today()
    paths = get_thumbnail_paths(start_date, end_date)
    
    for path in paths:
        if path[1] not in files:
            save_to_s3(path[0], path[1], s3, bucket_name)
    
    return
    

    

In [ ]:
get_thumbnails()

No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_211000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_210000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_205000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_204000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_203000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_202000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_201000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_200000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_195000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_1

No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_074000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_073000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_072000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_071000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_070000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_065000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_064000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_063000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_062000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_0

No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_195000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_194000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_193000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_192000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_191000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_190000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_185000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_184000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_183000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/16/2019_0516_1

No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_175000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_174000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_173000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_172000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_171000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_170000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_165000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_164000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_163000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/12/2019_0512_1

No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_204000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_203000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_202000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_201000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_200000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_195000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_194000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_193000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_192000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2019/05/11/2019_0511_1

...

No image found at https://spaceneedledev.com/panocam/assets/2017/09/07/2017_0907_071000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/07/2017_0907_070000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/07/2017_0907_065000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/07/2017_0907_052000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/07/2017_0907_051000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/07/2017_0907_050000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/06/2017_0906_211000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/06/2017_0906_210000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/06/2017_0906_205000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/09/06/2017_0906_2

No image found at https://spaceneedledev.com/panocam/assets/2017/08/28/2017_0828_101000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/28/2017_0828_100000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/28/2017_0828_051000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/28/2017_0828_050000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/27/2017_0827_211000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/27/2017_0827_210000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/27/2017_0827_051000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/27/2017_0827_050000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/26/2017_0826_211000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/26/2017_0826_0

No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_164000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_163000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_162000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_161000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_160000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_155000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_154000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_153000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_152000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/08/2017_0808_1

No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_114000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_113000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_112000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_111000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_110000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_105000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_104000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_103000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_102000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/08/07/2017_0807_1

No image found at https://spaceneedledev.com/panocam/assets/2017/07/06/2017_0706_164000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/06/2017_0706_163000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/04/2017_0704_123000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/04/2017_0704_122000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/03/2017_0703_060000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/03/2017_0703_055000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/03/2017_0703_054000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/03/2017_0703_053000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/01/2017_0701_193000/thumbnail.jpg
No image found at https://spaceneedledev.com/panocam/assets/2017/07/01/2017_0701_1

In [152]:
good_url = "https://spaceneedledev.com/panocam/assets/2019/05/30/2019_0530_093000/thumbnail.jpg"
good_fp = "2019/05/29/2019_0530_093000/thumbnail.jpg"
save_to_s3(good_url, good_fp, bucket_name)

In [ ]:
files = [file.key for file in bucket.objects.filter()]
print(len(files))
files

In [149]:
s3 = boto3.resource('s3')
s3.

In [140]:
good_url = "https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0530_093000/thumbnail.jpg"
good_fp = "2019/05/31/2019_0531_093000/thumbnail.jpg"
save_to_s3(good_url, good_fp, bucket_name)

Good to go. Found the bucket to upload the image into.
No image found at https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0530_093000/thumbnail.jpg


In [85]:
r = requests.get("https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_093000/thumbnail.jpg")
g = requests.get("https://spaceneedledev.com/panocam/assets/2019/05/30/2019_0530_093000/thumbnail.jpg")

In [158]:
r = requests.get("https://spaceneedledev.com/panocam/assets/2019/05/28/")

In [179]:
[l for l in r.iter_lines()]

[b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">',
 b'<html>',
 b' <head>',
 b'  <title>Index of /panocam/assets/2019/05/28</title>',
 b' </head>',
 b' <body>',
 b'<h1>Index of /panocam/assets/2019/05/28</h1>',
 b'<pre>      <a href="?C=N;O=D">Name</a>                    <a href="?C=M;O=A">Last modified</a>      <a href="?C=S;O=A">Size</a>  <a href="?C=D;O=A">Description</a><hr>      <a href="/panocam/assets/2019/05/">Parent Directory</a>                             -   ',
 b'      <a href="2019_0528_041000/">2019_0528_041000/</a>       28-May-2019 04:42    -   ',
 b'      <a href="2019_0528_042000/">2019_0528_042000/</a>       28-May-2019 04:42    -   ',
 b'      <a href="2019_0528_043000/">2019_0528_043000/</a>       28-May-2019 05:03    -   ',
 b'      <a href="2019_0528_044000/">2019_0528_044000/</a>       28-May-2019 05:03    -   ',
 b'      <a href="2019_0528_045000/">2019_0528_045000/</a>       28-May-2019 05:19    -   ',
 b'      <a href="2019_0528_050000/">2019_0528_

In [75]:
def get_thumbnail_urls(start_date, end_date):
    base = "https://spaceneedledev.com/panocam/assets/"
    tail = "thumbnail.jpg"
    get_url = lambda x: f'https://spaceneedledev.com/panocam/assets/{x}'

    bucket_name = 'eye-of-the-needle'
    
    num_days = (end_date - start_date).days
    date_list = [end_date - timedelta(days=x) for x in range(0, num_days)]
    times = [str(t).zfill(6) for t in range(211000, 50000-1, -1000) if int(str(t)[-4]) < 6]
    
    for day in date_list:
        y = day.year
        m = str(day.month).zfill(2)
        d = str(day.day).zfill(2)
        
        for t in times:
            filepath = f'{y}/{m}/{d}/{y}_{m}{d}_{t}/{tail}'
            url = get_url(filepath)
            #save_to_s3(url, filepath, bucket_name)
            print(f'Id save {url} to {filepath}')
    return

In [76]:
start_date = date(2019, 5, 1)
end_date = date.today()
get_thumbnail_urls(start_date, end_date)

Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_211000/thumbnail.jpg to 2019/05/31/2019_0531_211000/thumbnail.jpg
Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_210000/thumbnail.jpg to 2019/05/31/2019_0531_210000/thumbnail.jpg
Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_205000/thumbnail.jpg to 2019/05/31/2019_0531_205000/thumbnail.jpg
Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_204000/thumbnail.jpg to 2019/05/31/2019_0531_204000/thumbnail.jpg
Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_203000/thumbnail.jpg to 2019/05/31/2019_0531_203000/thumbnail.jpg
Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_202000/thumbnail.jpg to 2019/05/31/2019_0531_202000/thumbnail.jpg
Id save https://spaceneedledev.com/panocam/assets/2019/05/31/2019_0531_201000/thumbnail.jpg to 2019/05/31/2019_0531_201000/thumbnail.jpg
Id save https://spaceneedledev.com/panoca

In [46]:
from datetime import date, datetime, timedelta
start_date = date(2015, 1, 1)
end_date = date.today()
num_days = (end_date - start_date).days + 1
get_thumbnail_urls(start_date, end_date)

2019 5 31


In [49]:
date_list = [end_date - timedelta(days=x) for x in range(0, num_days + 1)]
str(date_list[3].month).zfill(2)


'05'

In [67]:
[str(t).zfill(6) for t in range(211000, 50000-1, -1000)]

'050000'

In [65]:
[t for t in range(211000, 50000-1, -1000)][-1]

50000

In [12]:
re.match('fetch("[.*]")', fetches)

In [53]:
bucket_name = 'eye-of-the-needle'
filepath = 'test_s3_image.jpg'
url = 'https://spaceneedledev.com/panocam/assets/2019/05/28/2019_0528_120000/thumbnail.jpg'
save_to_s3(url, filepath, bucket_name)

Good to go. Found the bucket to upload the image into.


# Scrub the Data

*Look through the raw data files and see what you will need to do to them in order to have a workable data set. If your source data is already well-formatted, you may want to ask yourself why it hasn't already been analyzed and what other people may have overlooked when they were working on it. Are there other data sources that might give you more insights on some of the data you have here?*

*The end goal of this step is to produce a [design matrix](https://en.wikipedia.org/wiki/Design_matrix), containing one column for every variable that you are modeling, including a column for the outputs, and one row for every observation in your data set. It needs to be in a format that won't cause any problems as you visualize and model your data.*

In [8]:
## %%writefile ../src/features/build_features.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/raw, cleans them,
    and converts the data into a design matrix that is ready for modeling.
    """
    # clean_dataset_1('data/raw', filename)
    # clean_dataset_2('data/raw', filename)
    # save_cleaned_data_1('data/interim', filename)
    # save_cleaned_data_2('data/interim', filename)
    # build_features()
    # save_features('data/processed')
    pass


*Before moving on to exploratory analysis, write down some notes about challenges encountered while working with this data that might be helpful for anyone else (including yourself) who may work through this later on.*

# Explore the Data

*Before you start exploring the data, write out your thought process about what you're looking for and what you expect to find. Take a minute to confirm that your plan actually makes sense.*

*Calculate summary statistics and plot some charts to give you an idea what types of useful relationships might be in your dataset. Use these insights to go back and download additional data or engineer new features if necessary. Not now though... remember we're still just trying to finish the MVP!*

In [ ]:
## %%writefile ../src/visualization/visualize.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # data = load_features('data/processed')
    # describe_features(data, 'reports/')
    # generate_charts(data, 'reports/figures/')
    pass


*What did you learn? What relationships do you think will be most helpful as you build your model?*

# Model the Data

*Describe the algorithm or algorithms that you plan to use to train with your data. How do these algorithms work? Why are they good choices for this data and problem space?*

In [ ]:
## %%writefile ../src/models/train_model.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # data = load_features('data/processed/')
    # train, test = train_test_split(data)
    # save_train_test(train, test, 'data/processed/')
    # model = build_model()
    # model.fit(train)
    # save_model(model, 'models/')
    pass


In [ ]:
## %%writefile ../src/models/predict_model.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # test_X, test_y = load_test_data('data/processed')
    # trained_model = load_model('models/')
    # predictions = trained_model.predict(test_X)
    # metrics = evaluate(test_y, predictions)
    # save_metrics('reports/')
    pass



_Write down any thoughts you may have about working with these algorithms on this data. What other ideas do you want to try out as you iterate on this pipeline?_

# Interpret the Model

_Write up the things you learned, and how well your model performed. Be sure address the model's strengths and weaknesses. What types of data does it handle well? What types of observations tend to give it a hard time? What future work would you or someone reading this might want to do, building on the lessons learned and tools developed in this project?_